In [1]:
import cv2
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import torch.nn.functional as F

import os
from ImageClassifierNets import LightweightCNN

/Users/jvelasquez/deepRL_py38/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load an image from path and convert it to a tensor
def load_image(path):
    img = cv2.imread(path)
    
    # Resize image to 160x120 using INTER_AREA interpolation
    img = cv2.resize(img, (160, 120), interpolation=cv2.INTER_AREA)

    # Convert image to RGB and normalize
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) 
    cv2.normalize(img_rgb, img_rgb, 0, 1, cv2.NORM_MINMAX)

    # Swap the dimensions and convert to tensor
    img_tensor = torch.as_tensor(img_rgb).permute(2, 0, 1)

    return img_tensor.unsqueeze(0)

# Load the model from path
def load_model(path):
    model = torch.load(path)

    return model

def make_prediction(model, img_tensor):
    model.eval()
    with torch.no_grad():
        # Run the model
        out = model(img_tensor)

        #print(out)

        # Retrieve the index of the highest activation
        _, pred = torch.max(out, 1)

        if pred == 0:
            return "Green & Fallen"
        elif pred == 1:
            return "Green & Standing"
        elif pred == 2:
            return "Red & Fallen"
        elif pred == 3:
            return "Red & Standing"
        elif pred == 4:
            return "White & Fallen"
        elif pred == 5:
            return "White & Standing"
        else:
            return "Error"
        
        

In [3]:
# Load the model
model = LightweightCNN()
model.load_state_dict(torch.load("/Users/jvelasquez/Virginia_Tech/Spring_2023/ECE_4806/ieee-robotics-2023-code/src/jetson_code/pedestal_classification/lightweight_net_color_orientation_v4.pth"))

<All keys matched successfully>

In [5]:
# Load the image
path = "/Users/jvelasquez/Downloads/pond/lying_down/"

# Go through all pictures in the path and make predictions
for filename in os.listdir(path):
    if filename.endswith(".jpg"):
        img_tensor = load_image(path + filename)
        print("Prediction for " + filename + ": " + make_prediction(model, img_tensor))
        continue
    else:
        continue


Prediction for nvcamtest_8544_s00_00018.jpg: White & Fallen
Prediction for nvcamtest_8544_s00_00024.jpg: White & Fallen
Prediction for nvcamtest_8544_s00_00030.jpg: White & Fallen
Prediction for nvcamtest_8619_s00_00098.jpg: White & Fallen
Prediction for nvcamtest_8619_s00_00067.jpg: White & Fallen
Prediction for nvcamtest_8619_s00_00073.jpg: White & Fallen
Prediction for nvcamtest_9115_s00_00050.jpg: Green & Fallen
Prediction for nvcamtest_9115_s00_00044.jpg: Green & Fallen
Prediction for nvcamtest_9115_s00_00078.jpg: Green & Fallen
Prediction for nvcamtest_9115_s00_00093.jpg: Green & Fallen
Prediction for nvcamtest_9115_s00_00087.jpg: Green & Fallen
Prediction for nvcamtest_9115_s00_00086.jpg: Green & Fallen
Prediction for nvcamtest_9115_s00_00092.jpg: Green & Fallen
Prediction for nvcamtest_9115_s00_00079.jpg: Green & Fallen
Prediction for nvcamtest_9115_s00_00045.jpg: Green & Fallen
Prediction for nvcamtest_9115_s00_00051.jpg: Green & Fallen
Prediction for nvcamtest_8619_s00_00072.